<a href="https://colab.research.google.com/github/SandipanMajhi/Document-Analysis/blob/main/Mnist_feature_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import numpy as np
import cv2
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
train_set = datasets.MNIST(root = "./data", train=True, download=True, transform=None)
test_set = datasets.MNIST(root = "./data", train=False, download = True, transform=None)

In [ ]:
train_arrays = []
train_labels = []
test_arrays = []
test_labels = []

def pil_to_np(stock_set):
  target_threshold = []
  target_set = []
  target_labels = []
  for i in range(0, len(stock_set)):
    img = np.array(stock_set[i][0], dtype = np.uint8)
    thresh , im_bw = cv2.threshold(img,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU) 
    target_threshold.append(thresh)
    target_set.append(np.array(im_bw, dtype = np.uint8))
    target_labels.append(np.array(stock_set[i][1]))

  return target_threshold, target_set, target_labels

train_thresh, train_arrays, train_labels = pil_to_np(train_set)
test_thresh, test_arrays, test_labels = pil_to_np(test_set)

In [ ]:
train_arrays[0].shape

(28, 28)

In [ ]:
### Creating contour maps from grayscale image

def gray_to_contours(dataset):
  contours = []
  for i in range(0, len(dataset)):
    contour, hierarchy = cv2.findContours(image=dataset[i], mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
    contour_transform = np.zeros((dataset[i].shape))
    contour_transform = cv2.drawContours(image = contour_transform, contours = contour , contourIdx=-1, color=(255))
    contours.append(contour_transform)
  
  return contours

train_contours = gray_to_contours(train_arrays)
test_contours = gray_to_contours(test_arrays)

In [ ]:
class Data_Maps():
  def __init__(self, dataset):
    self.dataset = dataset
      
  
  def map_diff(self, cmaps):
    maps = []
    for i in range(len(cmaps)):
      sz = len(cmaps[i])
      temp = cmaps[i][0]
      for j in range(0,len(cmaps[i])):
        cmaps[i][j] = (cmaps[i][j] - cmaps[i][(j+1)%sz])
      cmaps[i][-1] = (cmaps[i][-1] - temp)
    return cmaps
      
      


  def left_maps(self, rstep, cstep):
    sz = len(self.dataset)
    left_maps = []
    for i in range(sz):
      lmap = []
      for row in range(0,self.dataset[i].shape[0],rstep):
        count = 0
        for col in range(0,self.dataset[i].shape[1],cstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        lmap.append(count)
      left_maps.append(lmap)
    return left_maps

  def right_maps(self, rstep, cstep):
    sz = len(self.dataset)
    right_maps = []
    for i in range(sz):
      rmap = []
      for row in range(0,self.dataset[i].shape[0],rstep):
        count = 0
        for col in range(self.dataset[i].shape[1]-1,-1,-1 * cstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        rmap.append(count)
      right_maps.append(rmap)
    return right_maps

  def top_maps(self, rstep, cstep):
    sz = len(self.dataset)
    top_maps = []
    for i in range(sz):
      tmap = []
      for col in range(0,self.dataset[i].shape[1],cstep):
        count = 0
        for row in range(0,self.dataset[i].shape[0], rstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        tmap.append(count)
      top_maps.append(tmap)
    return top_maps
  
  def bottom_maps(self, rstep, cstep):
    sz = len(self.dataset)
    b_maps = []
    for i in range(sz):
      bmap = []
      for col in range(0,self.dataset[i].shape[0],cstep):
        count = 0
        for row in range(self.dataset[i].shape[1]-1,-1,-1 * rstep):
          if self.dataset[i][row][col] == 0:
            count = count + 1
          else:
            break
        bmap.append(count)
      b_maps.append(bmap)
    return b_maps

In [ ]:
### Testing
m = Data_Maps(train_arrays)



l_11 = m.left_maps(1,1)
r_11 = m.right_maps(1,1)
t_11 = m.top_maps(1,1)
b_11 = m.bottom_maps(1,1)

l_11d = m.map_diff(l_11)
r_11d = m.map_diff(r_11)
t_11d = m.map_diff(t_11)
b_11d = m.map_diff(b_11)

l_22 = m.left_maps(2,2)
r_22 = m.right_maps(2,2)
t_22 = m.top_maps(2,2)
b_22 = m.bottom_maps(2,2)

l_22d = m.map_diff(l_22)
r_22d = m.map_diff(r_22)
t_22d = m.map_diff(t_22)
b_22d = m.map_diff(b_22)

l_23 = m.left_maps(2,3)
r_23 = m.right_maps(2,3)
t_32 = m.top_maps(3,2)
b_32 = m.bottom_maps(3,2)

l_23d = m.map_diff(l_23)
r_23d = m.map_diff(r_23)
t_32d = m.map_diff(t_32)
b_32d = m.map_diff(b_32)

l_32 = m.left_maps(3,2)
r_32 = m.right_maps(3,2)
t_23 = m.top_maps(2,3)
b_23 = m.bottom_maps(2,3)

l_32d = m.map_diff(l_32)
r_32d = m.map_diff(r_32)
t_23d = m.map_diff(t_23)
b_23d = m.map_diff(b_23)

l_33 = m.left_maps(3,3)
r_33 = m.right_maps(3,3)
t_33 = m.top_maps(3,3)
b_33 = m.bottom_maps(3,3)

l_33d = m.map_diff(l_33)
r_33d = m.map_diff(r_33)
t_33d = m.map_diff(t_33)
b_33d = m.map_diff(b_33)
